In [1]:
import os
import numpy as np
import pywt
import random
import seaborn as sns
import tensorflow as tf
from keras.layers import Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (
  Input, Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D, 
  Dense, Dropout, GRU, Concatenate, LayerNormalization, MultiHeadAttention, 
  Reshape, Multiply, Softmax
)
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from utils import (
  encode_labels, check_gpu_availability, plot_loss_accuracytupian, 
  evaluate_model, plot_confusion_matrixtupian, plot_tsne, 
  plot_precision_recall_curve_multiclasstupian, plot_roc_curve_multiclasstupian, 
  AdjustLearningRateCallback, denoise2,count_labels,denoise2_iterative2,AdjustLearningRateCallback
)
from utils import plot_precision_recall_curve_multiclass,plot_roc_curve_multiclass2,calculate_g_mean,plot_confusion_matrix,plot_confusion_matrix2,plot_loss_accuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1DTranspose


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
check_gpu_availability()

GPU 可用


In [3]:
datafilename1 = "C:\\Users\\Administrator\\Desktop\\database\\cinc2017denoise.npz"
data1 = np.load(datafilename1, allow_pickle=True)
X_train, y_train, X_val, y_val, X_test, y_test = data1['ecgstrain'], data1['labelstrain'], data1['ecgsval'], data1['labelsval'], data1['ecgstest'], data1['labelstest']

In [4]:
y_train = encode_labels(y_train)
y_test = encode_labels(y_test)
y_val= encode_labels(y_val)
y_train = to_categorical(y_train, num_classes=4)
y_val=to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dropout, Dense,MaxPool1D
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size, strides):
    shortcut = x
    # 主卷积路径
    x = Conv1D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # 如果shortcut的形状与x不匹配，则调整shortcut
    if shortcut.shape[-1] != x.shape[-1] or shortcut.shape[-2] != x.shape[-2]:
        shortcut = Conv1D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # 添加跳过连接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 19, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = residual_block(x, 32, 3, 2)
    x = residual_block(x, 64, 3, 2)
    x = residual_block(x, 128, 3, 2)
    x1 = GRU(128, return_sequences=False)(x)
    x2 = GRU(128, return_sequences=False)(x)
    x=x1+x2
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Example usage:
model =resnet((4500, 1), 4)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4500, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 4500, 32)     640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 4500, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 4500, 32)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
callback = AdjustLearningRateCallback(factor=0.1, patience=2, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=256, epochs=80, validation_data=(X_val, y_val), callbacks=[callback,early_stopping])

Epoch 1/80
55/55 [==============================] - 31s 335ms/step - loss: 0.4207 - accuracy: 0.5794 - val_loss: 0.4360 - val_accuracy: 0.5760
Epoch 2/80
55/55 [==============================] - 17s 308ms/step - loss: 0.4028 - accuracy: 0.5942 - val_loss: 0.4113 - val_accuracy: 0.5815
Epoch 3/80
55/55 [==============================] - 17s 303ms/step - loss: 0.3933 - accuracy: 0.6067 - val_loss: 0.3959 - val_accuracy: 0.6060
Epoch 4/80
55/55 [==============================] - 17s 302ms/step - loss: 0.3691 - accuracy: 0.6436 - val_loss: 0.3698 - val_accuracy: 0.6381
Epoch 5/80
55/55 [==============================] - 17s 301ms/step - loss: 0.3367 - accuracy: 0.6777 - val_loss: 0.3393 - val_accuracy: 0.6741
Epoch 6/80
55/55 [==============================] - 17s 306ms/step - loss: 0.3106 - accuracy: 0.7127 - val_loss: 0.3280 - val_accuracy: 0.6937
Epoch 7/80
55/55 [==============================] - 17s 306ms/step - loss: 0.2847 - accuracy: 0.7493 - val_loss: 0.2920 - val_accuracy: 0.7450

In [7]:
import numpy as np

def add_low_frequency_noise_multidim(data, snr, frequency_range=(0, 5), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [8]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,0)

In [9]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.638237741672889
Recall: 0.6822066900741324
F1 Score: 0.6574919908350823
Accuracy: 0.7518101367658889
Class 1 - Precision: 0.4657039711191336, Recall: 0.5682819383259912, F1 Score: 0.511904761904762
Class 2 - Precision: 0.8568548387096774, Recall: 0.8494337108594271, F1 Score: 0.8531281365005018
Class 3 - Precision: 0.6470588235294118, Recall: 0.6111111111111112, F1 Score: 0.6285714285714287
Class 4 - Precision: 0.5833333333333334, Recall: 0.7, F1 Score: 0.6363636363636365
Class 1 Accuracy: 0.9010458567980691
Class 2 Accuracy: 0.8234111021721641
Class 3 Accuracy: 0.7855993563958166
Class 4 Accuracy: 0.9935639581657281


In [10]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,5)

In [11]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.8268610002435349
Recall: 0.7766360935488308
F1 Score: 0.79900227515049
Accuracy: 0.8382944489139179
Class 1 - Precision: 0.7955555555555556, Recall: 0.788546255506608, F1 Score: 0.7920353982300885
Class 2 - Precision: 0.87997432605905, Recall: 0.9133910726182545, F1 Score: 0.8963713631905851
Class 3 - Precision: 0.7569141193595342, Recall: 0.7046070460704607, F1 Score: 0.7298245614035087
Class 4 - Precision: 0.875, Recall: 0.7, F1 Score: 0.7777777777777777
Class 1 Accuracy: 0.9621882542236525
Class 2 Accuracy: 0.8724859211584876
Class 3 Accuracy: 0.8451327433628318
Class 4 Accuracy: 0.996781979082864


In [12]:
import numpy as np

def add_high_frequency_noise_multidim(data, snr, frequency_range=(0, 200), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [13]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,0)

In [14]:
evaluate_model(model,X_test_noisy,y_test)

Precision: 0.5788805359763913
Recall: 0.7016657998766753
F1 Score: 0.6134877160157673
Accuracy: 0.7477876106194691
Class 1 - Precision: 0.5020408163265306, Recall: 0.5418502202643172, F1 Score: 0.5211864406779662
Class 2 - Precision: 0.8522033898305085, Recall: 0.8374417055296469, F1 Score: 0.844758064516129
Class 3 - Precision: 0.6475524475524476, Recall: 0.6273712737127372, F1 Score: 0.6373021335168617
Class 4 - Precision: 0.3137254901960784, Recall: 0.8, F1 Score: 0.45070422535211263
Class 1 Accuracy: 0.9090909090909091
Class 2 Accuracy: 0.8141592920353983
Class 3 Accuracy: 0.7880128720836685
Class 4 Accuracy: 0.9843121480289622


In [15]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,5)

In [16]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.8320141311787147
Recall: 0.764621841232309
F1 Score: 0.791913461421289
Accuracy: 0.8499597747385358
Class 1 - Precision: 0.7991452991452992, Recall: 0.8237885462555066, F1 Score: 0.8112798264642083
Class 2 - Precision: 0.883248730964467, Recall: 0.927381745502998, F1 Score: 0.9047773805654858
Class 3 - Precision: 0.7885196374622356, Recall: 0.7073170731707317, F1 Score: 0.7457142857142857
Class 4 - Precision: 0.8571428571428571, Recall: 0.6, F1 Score: 0.7058823529411764
Class 1 Accuracy: 0.9650040225261464
Class 2 Accuracy: 0.8821399839098955
Class 3 Accuracy: 0.8567980691874497
Class 4 Accuracy: 0.99597747385358
